In [1]:
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pyspark.sql import SparkSession

In [26]:
def scrapData():
    #import requests
    import dateparser
    from datetime import datetime, timedelta
    #import pandas as pd
    #from bs4 import BeautifulSoup
    import hashlib
    import time
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from pyspark.sql import SparkSession
    from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, FloatType

    # Create SparkSession
    spark = SparkSession.builder \
    .appName("Save Scrapped data as CSV") \
    .getOrCreate()
    
    spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
    
    """
    "article_hash" TEXT,
    "article_title" TEXT,
    "category" TEXT,
    "publish_date" DATE,
    "article_body" TEXT,
    "raw_content" TEXT,
    "source_entity" TEXT,
    "article_link" TEXT,

    "generated_summary" TEXT,
    "negative_score" NUMERIC,
    "importance_score" NUMERIC,

    PRIMARY KEY (article_hash)
    """
    
    schema = StructType([
        StructField("article_hash", StringType(), nullable=True),
        StructField("article_title", StringType(), nullable=True),
        StructField("category", StringType(), nullable=True),
        StructField("publish_date", DateType(), nullable=True),
        StructField("article_body", StringType(), nullable=True),
        StructField("raw_content", StringType(), nullable=True),
        StructField("source_entity", StringType(), nullable=True),
        StructField("article_link", StringType(), nullable=True),
        
        StructField("generated_summary", StringType(), nullable=True),
        StructField("negative_score", FloatType(), nullable=True),
        StructField("importance_score", FloatType(), nullable=True),
    ])
    
    

    def hash_text(text):
        sha256_hash = hashlib.sha256()
        sha256_hash.update(text.encode('utf-8'))
        hashed_text = sha256_hash.hexdigest()
        return hashed_text

    def is_ready(browser):
        return browser.execute_script(r"""
            return document.readyState === 'complete'
        """)

    options = webdriver.ChromeOptions()
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    #browser = webdriver.Chrome('./chromedriver/',options=options)
    remote_webdriver = 'http://localhost'

    with webdriver.Remote(f'{remote_webdriver}:4444/wd/hub', options=options) as browser:
        print("1) BROWSER OK")
        # Load web page
        browser.get("https://www.biobiochile.cl/lista/categorias/nacional")
        print("2) GET OK")
        button = browser.find_element(By.CLASS_NAME, 'fetch-btn')
        print("3) BUTTON OK")
        # Date A: Now
        # Date B: Reference, example: Now - 24h, or, Last post date
        date_A = datetime.now()
        date_B = date_A - timedelta(hours=3)
        # TODO: Query last post date

        while(date_A > date_B):
            # Execute fetch/click function
            browser.execute_script("arguments[0].click();", button)   
            print("4) CLICK OK") 

            # Wait to load
            time.sleep(1)
            WebDriverWait(browser, 30).until(is_ready)

            # Search news
            elements = browser.find_elements(By.XPATH, "/html/body/main/div/section/div[2]/div[2]/div/article")
            print("5) FETCH OK") 

            # Fetched items are loaded in a different list
            fetch_elements = browser.find_elements(By.XPATH, "/html/body/main/div/section/div[2]/div[2]/div/div/div[1]/div")
            print("6) FETCH 2 OK") 

            # Get last fetched post datetime
            date_A = dateparser.parse(str(fetch_elements[len(fetch_elements)-1].find_element(By.CLASS_NAME, 'article-date-hour').text))

            articles = []
            
            if (date_A <= date_B):
                print("Loading data...")
                full_elements = elements + fetch_elements
                for elem in full_elements:
                    # title
                    title = str(elem.find_element(By.CLASS_NAME, 'article-title').text)
                    # link
                    link = str(elem.find_elements(By.TAG_NAME, 'a')[0].get_attribute('href'))
                    # date
                    #date_post = dateparser.parse(str(elem.find_element(By.CLASS_NAME, 'article-date-hour').text))

                    # visit link and get news body text
                    #article_data = getNewsInfo(str(elem.find_element(By.TAG_NAME, 'a').get_attribute('href')))
                    article_data = {}
                    article_data["article_hash"] = hash_text(link)
                    article_data["article_title"] = title
                    article_data["article_link"] = link
                    article_data["category"] = "Nacional"
                    article_data["source_entity"] = "biobiochile.cl"
                    articles.append(article_data)
                data_df = spark.createDataFrame(list(articles), schema)
                data_df.coalesce(1).write.option('header', True).csv("output.csv", header=True, mode="overwrite", sep=';', encoding="UTF-8", quote='"', escape='"')
        browser.close()
        spark.stop()
scrapData()

1) BROWSER OK
2) GET OK
3) BUTTON OK
4) CLICK OK
5) FETCH OK
6) FETCH 2 OK
Loading data...


In [2]:
URL = "https://www.biobiochile.cl/lista/categorias/nacional"
resp = requests.get(URL)
print(resp.status_code)
print(resp.text)

200
<!DOCTYPE html>
<html lang="es">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>BioBioChile - La Red de Prensa Más Grande de Chile</title>
    <link rel="preconnect" href="//busca.biobiochile.cl" />
    <link rel="preconnect" href="//media.biobiochile.cl" />
    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" />
    <link rel="apple-touch-icon" sizes="57x57" href="https://www.biobiochile.cl/assets/biobiochile/img/icons/apple-touch-icon-57x57.png" />
    <link rel="apple-touch-icon" sizes="60x60" href="https://www.biobiochile.cl/assets/biobiochile/img/icons/apple-touch-icon-60x60.png" />
    <link rel="apple-touch-icon" sizes="72x72" href="https://www.biobiochile.cl/assets/biobiochile/img/icons/apple-touch-icon-72x72.png" />
    <link rel="apple-touch-icon" sizes="76x76" href="https://www.biobiochil

In [48]:
soup = BeautifulSoup(resp.text, "lxml")
#class="article article-horizontal article-with-square justify-content-between"
elements = soup.select('[class^="article article-horizontal article-with-square justify-content-between"]')
#print(elements[0].a['href'], len(elements))
print(elements[1])

<article class="article article-horizontal article-with-square justify-content-between">
<div class="article-content-container">
<div class="article-text-container">
<a href="https://www.biobiochile.cl/noticias/nacional/chile/2023/07/05/vodanovic-ps-y-silencio-de-latorre-por-convenios-se-traiciono-la-confianza-a-nivel-personal.shtml">
<h2 class="article-title">Vodanovic (PS) y silencio de Latorre por convenios: "Se traicionó la confianza a nivel personal"</h2>
</a>
<div class="article-author-container">
                                            Por
                                            <a class="article-author" href="https://www.biobiochile.cl/lista/autores/fsolis">Francisco Solís</a>
</div>
<div class="article-date-hour">
                                            Miércoles 05 Julio, 2023 | 19:16                                        </div>
</div>
</div>
<a href="https://www.biobiochile.cl/noticias/nacional/chile/2023/07/05/vodanovic-ps-y-silencio-de-latorre-por-convenios-se

## BioBioChile

In [37]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import dateparser
import datetime

# Launch Chrome browser in headless mode
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--')
browser = webdriver.Chrome(options=options)
#browser2_links = webdriver.Chrome(options=options)
# Load web page
browser.get("https://www.biobiochile.cl/lista/categorias/nacional")

# Network transport takes time. Wait until the page is fully loaded
def is_ready(browser):
    return browser.execute_script(r"""
        return document.readyState === 'complete'
    """)

WebDriverWait(browser, 30).until(is_ready)

def getNewsInfo(link_href):
    #browser2_links.get(link_href)
    #WebDriverWait(browser2_links, 30).until(is_ready)
    #element = browser2_links.find_element(By.CLASS_NAME, 'post')
    #print(str(element.get_property('innerHTML')).strip())
    pass
"""    id_post = str(element.get_property('id')).split('-')[1]
    XPATH_article = f'//*[@id="post-{id_post}"]'
    element = browser2_links.find_element(By.XPATH, XPATH_article)
    post_image = element.find_element(By.CLASS_NAME, 'post-image')
    post_header = element.find_element(By.CLASS_NAME, 'post-excerpt')
    post_date = element.find_element(By.CLASS_NAME, 'post-date')
    print(post_header.text)
    print(dateparser.parse(str(post_date.text)))
    
    with open(f'imgs/image_{id_post}', 'wb') as file:
        file.write(post_image.screenshot_as_png)
        
    elements = element.find_elements(By.CLASS_NAME, f'banners-contenido-nota-{id_post} > p,h2')
    i=0
    for e in elements:
        i+=1
        print(i, e.text)"""

# Maximum search range, 24h
#last_date = dateparser.parse('2023-07-05 18:03:00')
last_date = datetime.datetime.now()
last_date_m_24h = last_date - datetime.timedelta(days=1)
i = 0
time.sleep(2)
WebDriverWait(browser, 100).until(is_ready)
# While last date is greater than last date minus 24 hours
button = browser.find_element(By.CLASS_NAME, 'fetch-btn')
print(button.text,)

while(last_date > last_date_m_24h):
    # Execute fetch function
    print('####################FETCHING DATA########################')

    browser.execute_script("arguments[0].click();", button)    
    
    time.sleep(2)
    WebDriverWait(browser, 100).until(is_ready)

    # Search for news headlines and print
    elements = browser.find_elements(By.XPATH, "/html/body/main/div/section/div[2]/div[2]/div/article")
    fetch_elements = browser.find_elements(By.XPATH, "/html/body/main/div/section/div[2]/div[2]/div/div/div[1]/div")

    #tags = browser.find_elements(By.TAG_NAME, 'a')
    print(len(elements))
    print(len(fetch_elements))
    
    #print(fetch_elements[len(fetch_elements)-1].find_element(By.CLASS_NAME, 'article-title').text)
    last_date = dateparser.parse(str(fetch_elements[len(fetch_elements)-1].find_element(By.CLASS_NAME, 'article-date-hour').text))
    #print(last_date)
    
    last_element = [fetch_elements[len(fetch_elements)-1]]
    
    for elem in last_element:
        print(elem.find_element(By.CLASS_NAME, 'article-title').text)
        print(elem.find_elements(By.TAG_NAME, 'a')[0].get_attribute('href'))
        last_date = dateparser.parse(str(elem.find_element(By.CLASS_NAME, 'article-date-hour').text))
        print(last_date)
        #getNewsInfo(str(elem.find_elements(By.TAG_NAME, 'a')[0].get_attribute('href')))
    
# Close the browser once finish
browser.close()

Cargar más
####################FETCHING DATA########################
19
10
Sismo de mediana magnitud se registra en región de Antofagasta
https://www.biobiochile.cl/noticias/nacional/region-de-antofagasta/2023/07/05/sismo-de-mediana-magnitud-se-registra-en-region-de-antofagasta-5.shtml
2023-07-05 20:45:00
####################FETCHING DATA########################
19
20
Jackson ordena sumario en Atacama: "Al menos un concurso no se ajustó a criterios de licitación"
https://www.biobiochile.cl/noticias/nacional/chile/2023/07/05/jackson-ordena-sumario-en-atacama-al-menos-un-concurso-no-se-ajusto-a-criterios-de-licitacion.shtml
2023-07-05 18:52:00
####################FETCHING DATA########################
19
30
Alcaldesa exige agilizar reparación de Estadio Municipal de San Antonio: retraso supera los 1.300 días
https://www.biobiochile.cl/noticias/nacional/region-de-valparaiso/2023/07/05/alcaldesa-exige-agilizar-reparacion-de-estadio-municipal-de-san-antonio-retraso-supera-los-1-300-dias.shtm

## La Tercera

In [41]:
URL = "https://www.latercera.com/categoria/nacional/page/2"
resp = requests.get(URL)
print(resp.status_code)
print(resp.text)

200
<!DOCTYPE html><html><head><title>Categoría: NACIONAL - La Tercera</title><meta http-equiv="refresh" content="300"/><script>var digitalData={"chartbeat_uid":"14295","chartbeat_domain":"latercera.com","siteDomainURL":"https://www.latercera.com","section":["nacional"],"tag":"","author":"","pageInstanceID":"archivo","page":{"pageInfo":{"pageID":"archivo","storyTitle":"","pageName":"/categoria/nacional","destinationURL":"https://www.latercera.com/categoria/nacional","author":"","issueDate":"","effectiveDate":"","publisher":"latercera.com","tags":[""]},"category":{"primaryCategory":"nacional","subCategory1":"","pageType":""},"attributes":{"country":"CL","language":"es-cl","server":"latercera_prod"}}}</script><script>dataLayer=[{}];</script><script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
        new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
        j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
        'https://www.goo

In [42]:
soup = BeautifulSoup(resp.text, "lxml")
#class="article article-horizontal article-with-square justify-content-between"
elements = soup.select('[class^="card | border-bottom float"]')
print(elements[0].h3.a.text, len(elements))

  Nacional  Sistema frontal: Senapred reporta 13.379 personas damnificadas, 1.632 albergados y 12.314 aislados 15


In [43]:
# Launch Chrome browser in headless mode
options = webdriver.ChromeOptions()
options.add_argument("headless")
browser = webdriver.Chrome(options=options)

for i in range(7):
    # Load web page
    browser.get(f"https://www.latercera.com/categoria/nacional/page/{i+1}")

    # Network transport takes time. Wait until the page is fully loaded
    def is_ready(browser):
        return browser.execute_script(r"""
            return document.readyState === 'complete'
        """)

    WebDriverWait(browser, 30).until(is_ready)
    # Execute fetch function
    #browser.execute_script("fetch()")
    time.sleep(1)
    WebDriverWait(browser, 30).until(is_ready)

    # Search for news headlines and print
    elements = browser.find_elements(By.XPATH, "/html/body/div/div[1]/div/main/section[2]/div[1]/article")

    for elem in elements:
        print(i, elem.text)

# Close the browser once finish
browser.close()

0 Nacional Presidente de la Suprema recorrió Licantén para verificar estado del tribunal y situación de funcionarios judiciales afectados por inundaciones
Se instruyó al Departamento de Bienestar del Poder Judicial realizar un catastro de la situación e iniciar una campaña de ayuda solidaria a nivel nacional.
José Navarrete Hace 1 hora
0 Nacional Boric valora labor de FF.AA. en emergencia: “Los militares están desplegados, acá no es que los militares estén en sus cuarteles”
Mandatario visitó la localidad de Ralco esta jornada. “Quizás no se ha llegado específicamente a todas las casas, a todos los lugares, pero se está haciendo el esfuerzo de despliegue y coordinación, a través de Senapred, para llegar a todas partes", afirmó.
José Navarrete Hace 2 horas
0 Nacional Colegio de Profesores se reúne con Mineduc y ratifica paro nacional para el próximo 26 de julio
Si no hay respuestas al petitorio, desde el gremio advierten más jornadas de paro, las que podrían culminar en una suspensión de

2 Nacional Transportes informa que este martes EFE restablecerá el servicio de trenes hasta Rancagua
El ministro de Transportes adelantó que el servicio hasta San Fernando se encuentra monitoreado y que "si todo sale bien ya podríamos tener el servicio hasta San Fernando desde Alameda para el día miércoles".
Shelmmy Carvajal 26 jun 2023 02:28 PM
2 Nacional Interrumpen suministro de gas por falla de gasoducto en Chillán
De acuerdo con lo informado por el seremi de Energía de Ñuble, Ricardo León, la compañía “detectó una caída de presión en el ducto que viene desde Argentina hacia Chillán. Como medida preventiva, la empresa procedió a cerrar el paso del ducto”.
Paz Rubio 26 jun 2023 01:58 PM
2 Nacional Presidente Boric llega hasta Licantén para visitar a damnificados por el sistema frontal
Se espera que el Mandatario visite el Hospital de Licantén, que quedó completamente afectado tras el paso de las lluvias.
Valentina Sánchez 26 jun 2023 01:41 PM
2 Nacional Gobierno anuncia que Bono de 

3 Nacional EFE mantiene suspensión de trenes hacia Rancagua y Chillán por daños en las vías
Desde la estatal señalaron que la suspensión de las operaciones se mantendrá por al menos cinco días “pues se realizarán evaluaciones en el entorno del puente ferroviario sobre el río Maipo, considerando afectación a infraestructuras que podrían comprometer la operación de los trenes”.
Valentina Sánchez 25 jun 2023 10:08 AM
4 Nacional Robo frustrado a empresa de chocolates en Recoleta: hay cuatro detenidos
Conforme a la información que entregó el dueño del local, no lograron sustraer ninguna especie desde el local comercial.
Ariadna Rodríguez 25 jun 2023 09:30 AM
4 Nacional Meteorología emite alerta por tornados y trombas marinas en Los Lagos
Además, el organismo pronosticó la ocurrencia de “probables tormentas eléctricas” en los sectores de litoral, cordillera de la costa, valle, Chiloé y litoral interior de la región".
Valentina Sánchez 25 jun 2023 09:18 AM
4 Nacional Conductor fallece tras ch

5 Nacional Sistema frontal: 3 desaparecidos, más de 2700 personas aisladas y 253 viviendas con daños
La ministra del Interior, Carolina Tohá, informó además que "hay preocupación por pacientes que están en situación delicada de salud y se está trabajando para hacer el rescate en esos lugares".
Ariadna Rodríguez 24 jun 2023 11:00 AM
6 Nacional Carabineros rescata a dos personas atrapadas en el sector El Toyo: Intensas lluvias dificultaron la búsqueda
La pareja se encontraba desde el pasado lunes acampando en el sector El Toyo, San José de Maipo, pero producto de las intensas lluvias, al intentar regresar, se extraviaron.
Shelmmy Carvajal 24 jun 2023 10:40 AM
6 Nacional EFE mantiene suspensión del servicio por sistema frontal y como medida preventiva de seguridad
Desde la estatal informaron que el servicio para mañana domingo está condicionado al el estado de las vías y al frente climático.
Valentina Sánchez 24 jun 2023 10:18 AM
6 Nacional Reportan caída de vehículo en socavón de El Bosq

## 24 horas

In [45]:
URL = "https://www.24horas.cl/actualidad/nacional/p/1"
resp = requests.get(URL)
print(resp.status_code)
print(resp.text)

200
<!DOCTYPE HTML>
<html lang="es-CL">
<head>


<title>
        
             Nacional 
            
         | 24horas </title>
<meta name="robots" content="index,follow">
<meta charset="utf-8">
<meta name="description" content="">

<script src="/cdn-cgi/apps/head/vVjKV8B75pDt2jpS7YpoG_N2c8g.js"></script><link rel="canonical" href="">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<meta name="HandheldFriendly" content="True">
<meta name="format-detection" content="telephone=no">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="author" content="24horas ">
<link rel="shorcut icon" href="/favicon.ico" type="image/x-icon">
<link rel="icon" href="/favicon.ico" type="image/x-icon">

<meta name="twitter:card" content="summary" />
<meta name="twitter:site" content="website" />
<meta name="twitter:title" content="" />
<meta name="twitter:description" content="" />
<meta property="twitter:image" content="">
<meta name="twitter:site" content="">


<meta 

In [50]:
soup = BeautifulSoup(resp.text, "lxml")
#class="article article-horizontal article-with-square justify-content-between"
elements = soup.select('article[class^="col xs-12 xsm-12"]')
print(elements[0], len(elements))

<article class="col xs-12 xsm-12 sm-12 md-12 lg-12">
<div class="block-full">
<div class="box-dest box-horizontal box-lg">
<figure class="img-wrap">
<a href="/actualidad/nacional/anticonceptivo-marilow-es-retirado-del-mercado-alerta-isp">
<img alt="" class="lazy" data-src="/24horas/site/artic/20230627/imag/foto_0000001320230627204732/Aton_500229.jpg" src="/24horas/imag/v1/default/default_600x338.jpg"/>
</a>
</figure>
<div class="cont-txt">
<a href="/actualidad/nacional/anticonceptivo-marilow-es-retirado-del-mercado-alerta-isp">
<p class="fecha">Martes 27 de junio de 2023</p>
<h3 class="tit">Retiran anticonceptivo del mercado tras alerta del ISP: "Afecta a todos los lotes vigentes del producto"</h3>
<p class="baj">Ante la alerta emitida por Marilow, el Instituto de Salud Pública recomendó utilizar otros métodos anticonceptivos adicionales, como preservativos al menos durante 1 mes, mientras realiza el cambio de marca. </p>
</a>
</div>
</div>
</div>
</article> 9


In [51]:
# Launch Chrome browser in headless mode
options = webdriver.ChromeOptions()
options.add_argument("headless")
browser = webdriver.Chrome(options=options)

for i in range(7):
    # Load web page
    browser.get(f"https://www.24horas.cl/actualidad/nacional/p/{i+1}")

    # Network transport takes time. Wait until the page is fully loaded
    def is_ready(browser):
        return browser.execute_script(r"""
            return document.readyState === 'complete'
        """)

    WebDriverWait(browser, 30).until(is_ready)
    # Execute fetch function
    #browser.execute_script("fetch()")
    time.sleep(1)
    WebDriverWait(browser, 30).until(is_ready)

    # Search for news headlines and print
    elements = browser.find_elements(By.XPATH, "/html/body/div[5]/section/div/div[1]/article")

    for elem in elements:
        print(i, elem.text)

# Close the browser once finish
browser.close()

1 Martes 27 de junio de 2023
¿Cuánto llovió? Zonas con casi 600 milímetros marcan balance por sistema frontal
Senapred entregó un balance de la cantidad de agua caída desde el inicio de las intensas precipitaciones que dejan miles de personas damnificadas.
1 Martes 27 de junio de 2023
Un fallecido tras accidente entre bus y dos vehículos en Paine
Según información policial, en el bus iban 33 pasajeros de los cuales "ninguno resultó lesionado".
1 Martes 27 de junio de 2023
Millonario robo en bodega de alimentos para mascotas: ladrones hicieron un forado en sitio colindante
El propietario del local aseguró que es la cuarta vez que le roban bajo el mismo modus operandi: "Se me ocurre que son las mismas personas, porque la forma de hacerlo es idéntica".
1 Martes 27 de junio de 2023
Senador Flores fue operado nuevamente tras accidente automovilístico que sufrió en 2022
El parlamentario se volcó en Pudahuel junto a su escolta y resultó con lesiones de diversa consideración, siendo la más gra

5 Domingo 25 de junio de 2023
Iba a exceso de velocidad: automovilista fallece tras chocar con bus del sistema RED
El hecho ocurrió en San Bernardo y el operador sólo se percató del impacto al perder el control de la máquina.
5 Sábado 24 de junio de 2023
Mapocho: ¿hay riesgo de desbordes o derrumbes en el río capitalino?
El río Mapocho vivió al límite los momentos más duros de estas lluvias, pero por lo pronto no debiera ocurrir un evento como el del año '82, cuando sencillamente se desbordó. Sin embargo, especialistas llaman a estar alerta, porque si se generan nuevas lluvias en la cordillera, los riesgos de derrumbes o desbordes aumentan de inmediato en algunos lugares de su cauce.
5 Sábado 24 de junio de 2023
Miles de viviendas dañadas en la zona centro sur
Hace tres décadas que no llovía con tanta intensidad en la zona central del país. La crecida de los ríos sigue provocando desbordes, dejando viviendas anegadas, familias aisladas, rutas y puentes cortados, personas desaparecidas 